# Error Analysis

This notebook, takes the predictions and gound truth files to identify the incorrect predictions and makes a combined image of the incorrect prediction (on the right), the ground truth (on the left) and the Full Phrase text (in black with red background)

In [1]:
predictions = []
test_data = []
test_labels = []

Reading the prediction and ground truth files

In [2]:
pred_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/pred_test_FullSentence.txt"
data_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/test_data.txt"
label_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/test_label.txt"
with open(pred_path) as f:
    predictions = f.readlines()

with open(data_path) as f:
    test_data = f.readlines()

with open(label_path) as f:
    test_labels = f.readlines()


In [3]:
predictions = [x.split("\t")[0] for x in predictions]


In [4]:
predictions[0]

'image.2166.jpg'

In [5]:
# test_data = [x.split("\t")[1] for x in test_data ]
temp = []
for t in test_data:
    tt = t.split("\t")
    try:
        ttt = tt[1]
        temp.append(ttt)
    except:
        print(tt)

In [6]:
test_data = temp

In [7]:
test_data[0]

'football goal'

In [8]:
test_labels = [x.strip("\n") for x in test_labels]

In [9]:
test_labels[0]

'image.2166.jpg'

In [10]:
# these indexes had nan values for Full Phrase, thus they were omitted
# test_labels.pop(40)
# test_labels.pop(367)
# test_data.pop(40)
# test_data.pop(367)

'hair bun'

Function to take in input images and text and make a collage to have a comparative view

In [13]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
listofimages=['Image2.jpg', 'Image6.jpg']

def create_collage(width, height, listofimages, img_name):
    img_name = img_name.replace(" ","-")
    cols = 2
    rows = 1
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    for p in listofimages:
        im = Image.open(p)
        im.thumbnail(size)
        ims.append(im)
    i = 0
    x = 0
    y = 0
    for col in range(cols):
        for row in range(rows):
            # print(i, x, y)
            new_im.paste(ims[i], (x, y))
            i += 1
            y += thumbnail_height
        x += thumbnail_width
        y = 0
    draw = ImageDraw.Draw(new_im)
    # draw.text(, img_name, fill=(255, 0, 0))
    position = (200, 180)
    left, top, right, bottom = draw.textbbox(position, img_name)
    draw.rectangle((left-5, top-5, right+5, bottom+5), fill="red")
    draw.text(position, img_name, fill="black",stroke_width=50)
    new_im.save('/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/errors/'+img_name+".jpg")

# create_collage(450, 200, listofimages)

Reading predictions and making collages for the incorrect predictions

In [15]:
test_img_path = '/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/semeval-2023-task-1-V-WSD-train-v1/val_v1/test_images_resized/'
for i,n in enumerate(predictions):
    if predictions[i] != test_labels[i]:
        # print(predictions[i])
        # print(test_labels[i])
        # print(test_data[i])
        create_collage(450, 200, [test_img_path+predictions[i],test_img_path+test_labels[i]],test_data[i])